### Using Decorators to Create Context Managers using Generator Functions

In [1]:
def open_file(fname, mode='r'):
    print('opening file...')
    f = open(fname, mode)
    try:
        yield f
    finally:
        print('closing file...')
        f.close()

In [2]:
class GenContextManager:
    def __init__(self, gen):
        self.gen = gen

    def __enter__(self):
        print('calling next to get the yielded value from generator')
        return next(self.gen)

    def __exit__(self, exc_type, exc_value, exc_tb):
        print('calling next to perform cleanup in generator')
        try:
            next(self.gen)
        except StopIteration:
            pass
        return False

In [3]:
file_gen = open_file('test.txt', 'w')
with GenContextManager(file_gen) as f:
    f.writelines('Sir Spamalot')

calling next to get the yielded value from generator
opening file...
calling next to perform cleanup in generator
closing file...


In [4]:
def context_manager_dec(gen_fn):
    def helper(*args, **kwargs):
        gen = gen_fn(*args, **kwargs)
        ctx = GenContextManager(gen)
        return ctx
    return helper

In [5]:
@context_manager_dec
def open_file(fname, mode='r'):
    print('opening file...')
    f = open(fname, mode)
    try:
        yield f
    finally:
        print('closing file...')
        f.close()
# same as
# open_file = context_manager_dec(open_file)

In [6]:
with open_file('test.txt') as f:
    print(f.readlines())

calling next to get the yielded value from generator
opening file...
['Sir Spamalot']
calling next to perform cleanup in generator
closing file...


In [7]:
from contextlib import contextmanager

In [8]:
@contextmanager
def open_file(fname, mode='r'):
    print('opening file...')
    f = open(fname, mode)
    try:
        yield f
    finally:
        print('closing file...')
        f.close()

In [9]:
with open_file('test.txt', 'r') as f:
    print(f.readlines())

opening file...
['Sir Spamalot']
closing file...


In [10]:
from time import perf_counter, sleep

In [13]:
@contextmanager
def timer():
    stats = dict()
    start = perf_counter()
    stats['start'] = start
    try:
        yield stats
    finally:
        end = perf_counter()
        stats['end'] = end
        stats['elapsed'] = end - start

In [14]:
with timer() as stats:
    sleep(2)


In [15]:
print(stats)

{'start': 1139.274633728, 'end': 1141.274837711, 'elapsed': 2.000203982999892}


In [16]:
import sys

In [17]:
@contextmanager
def out_to_file(fname):
    current_stdout = sys.stdout
    file = open(fname, 'w')
    sys.stdout = file
    try:
        yield
    finally:
        file.close()
        sys.stdout = current_stdout

In [18]:
with out_to_file('test.txt'):
    print('line 1')
    print('line 2')

In [19]:
print('hello')

hello


In [20]:
with open('test.txt', 'r') as f:
    print(f.readlines())

['line 1\n', 'line 2\n']


In [23]:
from contextlib import redirect_stdout

In [24]:
with open('test.txt', 'w') as f:
    with redirect_stdout(f):
        print('Look on the bright side of life')

In [25]:
with open('test.txt', 'r') as f:
    print(f.readlines())

['Look on the bright side of life\n']
